In [39]:
import tensorflow as tf
import pandas as pd
import shap

import os, sys
sys.path.append( os.path.join('..', '..', 'src', 'mlspec_flow') )
#from utils.predictor import PredictionModel
from utils.paths import bigquery_model_results_table, saved_model_dir

In [40]:
class PredictionModel:
    def __init__(self, flow_dir, model_name, signature="predict"):
        self.model        = tf.saved_model.load(saved_model_dir(flow_dir, model_name))
        self.predictor_fn = self.model.signatures[signature]
        
    def signature(self):
        return self.predictor_fn.structured_input_signature
    
    def predict(self, **kwargs):
        pred_input = {k:tf.constant([v]) for k,v in kwargs.items()}
        return self.predictor_fn(**pred_input)
    
    def predict_batch(self, batch):
        pred_input = {k:tf.constant(v) for k,v in batch.items()}
        return self.predictor_fn(**pred_input)
    
    def predict_records(self, records):
        return self.predict_batch(pd.DataFrame(records).to_dict(orient="list"))

In [41]:
flow_dir = 'TaxiTripTotalReduced2017FullV02'
model    = '20210518'
label    = "trip_total"

In [42]:
pred_model=PredictionModel(flow_dir, model)
explainer = shap.Explainer(pred_model.predict_records)

In [25]:
table = bigquery_model_results_table(flow_dir, model).replace(":",".")
df=pd.read_gbq(f"""select * except(unique_key, split_set, baseline, prediction) from `{table}` 
where ABS(MOD(FARM_FINGERPRINT(cast(unique_key as STRING)), 1000000)) = 1
""")

In [48]:
X=df.to_dict(orient="record")
shap_values = explainer(df)

TypeError: 'NoneType' object is not callable

In [23]:
print(f"""select * except(unique_key, split_set, baseline, prediction) from `{table}` 
where ABS(MOD(FARM_FINGERPRINT(cast(unique_key as STRING)), 1000)) = 1
""")

select * except(unique_key, split_set, baseline, prediction) from `mlteam-ml-specialization-2021.models_results.TaxiTripTotalReduced2017FullV02_20210518` 
where ABS(MOD(FARM_FINGERPRINT(cast(unique_key as STRING)), 1000)) = 1



In [34]:
data = [{"a":1, "b":2, "c":3},{"a":4, "b":5, "c":6}]
pd.DataFrame(data).to_dict(orient="list")

{'a': [1, 4], 'b': [2, 5], 'c': [3, 6]}